All of these instructions are available in the TOM Toolkit [Getting Started guide](https://tomtoolkit.github.io/docs/getting_started).

The following will set up our initial Django project with the necessary dependencies installed:

In [ ]:
pip install numpy
pip install dataclasses  # For Python 3.6 or earlier
django-admin startproject tom_demo
cd tom_demo
python3 -m venv env/
source env/bin/activate
pip install tomtoolkit
pip install jupyterlab

Next, we need to update our `INSTALLED_APPS` in settings.py, which should be in tom_demo/tom_demo. Add `tom_setup` to the bottom of the list.

In [ ]:
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'tom_setup',
]

Now we can run the setup script to install the TOM Toolkit with your preferred settings.

In [ ]:
./manage.py tom_setup

Finally, just run the necessary database migrations:

In [ ]:
./manage.py migrate

At this point, you can run your sample TOM with `./manage.py runserver`. 

In a separate terminal, make sure you start your virtual environment and you can start your Jupyter notebook server as follows:

In [ ]:
./manage.py shell_plus --notebook

Now we can start to experiment with the MARS Broker class.

First, we import the necessary TOM Toolkit classes:

In [10]:
from tom_alerts.brokers.mars import MARSQueryForm, MARSBroker
from tom_alerts.models import BrokerQuery

We can instantiate the `MARSQueryForm` class with the desired parameters, and calling `is_valid()` ensures that our datatypes are consistent with what's specified in the form class. As you can see, we end up with a dictionary of our query parameters.

In [20]:
mars_form = MARSQueryForm({
    'time__since': 864000,
    'objectcone': 'm101, 5',
    'rb__gte': 0.95
})
mars_form.is_valid()
print(mars_form.cleaned_data)

{'time__gt': '', 'time__lt': '', 'time__since': 864000, 'jd__gt': None, 'jd__lt': None, 'filter': '', 'cone': '', 'objectcone': 'm101, 5', 'objectidps': '', 'ra__gt': None, 'ra__lt': None, 'dec__gt': None, 'dec__lt': None, 'l__gt': None, 'l__lt': None, 'b__gt': None, 'b__lt': None, 'magpsf__gte': None, 'magpsf__lte': None, 'sigmapsf__lte': None, 'magap__gte': None, 'magap__lte': None, 'distnr__gte': None, 'distnr__lte': None, 'deltamaglatest__gte': None, 'deltamaglatest__lte': None, 'deltamagref__gte': None, 'deltamagref__lte': None, 'rb__gte': 0.95, 'drb__gte': None, 'classtar__gte': None, 'classtar__lte': None, 'fwhm__lte': None}


Now, we can go ahead and run the query on MARS. With our `MARSQueryForm` instance, it's as simple as instantiating the `MARSBroker` and calling `fetch_alerts()` on the cleaned data.

In [21]:
mars = MARSBroker()

alerts = mars.fetch_alerts(mars_form.cleaned_data)

print(sum(1 for _ in alerts))

38


With the TOM Toolkit, we can also store our query in our TOM as a `BrokerQuery` object with just a few parameters--a name for the query, the name of the `MARSBroker`, and the query parameters.

In [22]:
query = BrokerQuery.objects.create(
    name='Test Query',
    broker=mars.name,
    parameters=mars_form.serialize_parameters()
)

It's now trivial to simply get the query out of the database, and run it again on MARS:

In [23]:
query = BrokerQuery.objects.filter(name='Test Query', broker=mars.name)[0]

alerts = mars.fetch_alerts(query.parameters_as_dict)
print(sum(1 for _ in alerts))

38


With the available alerts from a query, we can convert each of them to a TOM `Target` object and store them in our local TOM DB.

In [24]:
alerts = mars.fetch_alerts(query.parameters_as_dict)
for alert in alerts:
    mars.to_target(alert)

Target post save hook: ZTF18aaigpcr created: True
Target post save hook: ZTF18aarqemn created: True
Target post save hook: ZTF18aaquezq created: True
Target post save hook: ZTF18aalrabq created: True
Target post save hook: ZTF18aajpaft created: True
Target post save hook: ZTF18aauubtt created: True
Target post save hook: ZTF19abovstj created: True
Target post save hook: ZTF18aauuaje created: True
Target post save hook: ZTF18aaigpcr created: True
Target post save hook: ZTF18aaigpcr created: True
Target post save hook: ZTF18aaigpcr created: True
Target post save hook: ZTF18aaavyjj created: True
Target post save hook: ZTF18aaigozk created: True
Target post save hook: ZTF17aabxpjd created: True
Target post save hook: ZTF18aarqemn created: True
Target post save hook: ZTF18aajpoup created: True
Target post save hook: ZTF18aakqmeq created: True
Target post save hook: ZTF18aaitnkd created: True
Target post save hook: ZTF18aalrabq created: True
Target post save hook: ZTF18aaqczkb created: True


And as expected, we now have as many `Target` objects in our TOM as we had alerts, which can be used for later reference.

In [25]:
from tom_targets.models import Target

print(len(Target.objects.all()))

38
